<a href="https://colab.research.google.com/github/alkang53/CNN-BASED-TEXT-CLASSIFICATION/blob/main/fromcsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from scipy.io import arff
import pandas as pd
import csv

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling2D, Dropout, Input, Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

drive.mount('/content/drive')

In [ ]:
#TTC4900 - kelimelerin orj halini içeren csv
csv_file = '/content/drive/My Drive/7allV03.csv'
data = pd.read_csv(csv_file)

corpus = data['text']
categoryNum = 7

In [ ]:
#TTC4900 - kelimelerin köklerini içeren csv
csv_file = '/content/drive/My Drive/7allV03_lem.csv'
data = pd.read_csv(csv_file)

corpus = data['text']
categoryNum = 7

In [ ]:
#müşteri yorumları - kelimelerin orj halini içeren csv
csv_file = '/content/drive/My Drive/e-ticaret_urun_yorumlari_as.csv'
data = pd.read_csv(csv_file, sep=';')

corpus = data['text']
categoryNum = 2

In [ ]:
#müşteri yorumları - kelimelerin köklerini içeren csv
csv_file = '/content/drive/My Drive/e-ticaret_urun_yorumlari_lem.csv'
data = pd.read_csv(csv_file, sep=';')

corpus = data['text']
categoryNum = 2

In [ ]:
#+ Stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

for indexstop, row in enumerate(corpus):
  tokens = word_tokenize(str(row))
  filtered_text = [t for t in tokens if not t.lower() in stopwords.words("turkish")]
  corpus[indexstop] = " ".join(filtered_text)
  #corpus.replace(str(row), " ".join(filtered_text))



In [ ]:
possible_labels = data.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
data['label'] = data.category.replace(label_dict)



In [ ]:
#TFIDF
vectorizer = TfidfVectorizer()
xBig = vectorizer.fit_transform(corpus)
y = data['label'].values



In [ ]:
#selectKbest
#Kütüphaneleri yükleme
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#Ki kare testine göre en iyi sonuç veren 2 özelliği seç
features = SelectKBest(chi2, k = 2000)
#features = SelectKBest(k = 2000)
x = features.fit_transform(xBig, y)
#Azaltılmış Özellikler
print('Orjinal Özellik Sayısı:', xBig.shape[1])
print('Seçilmiş Özellik Sayısı:', x.shape[1])

In [ ]:
#################naive bayes#############################
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  modelNB = MultinomialNB()

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelNB.fit(x[train],y[train])

  start = time.process_time();
  y_pred = modelNB.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

In [ ]:
#################random forest#############################

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  modelRF = RandomForestClassifier(n_estimators=100)

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelRF.fit(x[train],y[train])

  start = time.process_time();
  y_pred = modelRF.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

In [ ]:
#################  DVM  #############################

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  model = svm.SVC(kernel='linear') # Linear Kernel

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = model.fit(x[train],y[train])

  start = time.process_time();
  y_pred = model.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

In [ ]:
#################  knn  #############################

from sklearn.neighbors import KNeighborsClassifier

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x, y):
  #veri seti: 5692 öznitelikten oluşan bir arff dosyası
  modelKN = KNeighborsClassifier(n_neighbors = 5) # Linear Kernel

  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelKN.fit(x[train],y[train])

  start = time.process_time();
  y_pred = modelKN.predict(x[test]);
  end = time.process_time();
  ac=accuracy_score(y[test],y_pred);
  acc_per_fold.append(ac);

  cm = confusion_matrix(y[test], y_pred);

  
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

In [ ]:
x2 = x.toarray()
#x2 = x2.reshape(x2.shape[0], x2.shape[1], 1)
max_length=x2.shape[1]
vocab_size = x2.shape[1]

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []
prec_per_fold = []
recall_per_fold = []
f1_per_fold = []
confmat_per_fold = np.zeros((categoryNum, categoryNum),dtype=int)
predictTimeTotal = 0;
for train, test in kfold.split(x2, y):
  #veri seti: vocab_size öznitelikten oluşan bir arff dosyası
  modelCNN = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(128, 1, activation="relu", input_shape=(vocab_size,1)), #dropout 0.2 91,5 droopout 0.05 88 dropout 0.3 89
      tf.keras.layers.Flatten(input_shape=(vocab_size,1)),
      tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(7)
      ])


  modelCNN.summary()
  #define loss function variable
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  
  # compile the model
  modelCNN.compile(optimizer='adam',
             loss=loss_fn,
             metrics=['accuracy'])


  # Generate a print
  print("--")
  print(f'Training for fold {fold_no} ...')
  #train the model
  history = modelCNN.fit(x2[train],y[train],epochs=10)
  scores = modelCNN.evaluate(x2[test],y[test],verbose=2, batch_size=10)
  print(f'Score for fold {fold_no}: {modelCNN.metrics_names[0]} of {scores[0]}; {modelCNN.metrics_names[1]} of {scores[1]}')
  acc_per_fold.append(scores[1])
  loss_per_fold.append(scores[0])
  
  start = time.process_time();
  y_pred1 = modelCNN.predict(x2[test])
  end = time.process_time();
  print("predictTime: ", end - start)
  y_pred = np.argmax(y_pred1, axis=1)
  # Print f1, precision, and recall scores
  precision_score_ = precision_score(y[test], y_pred , average="macro")
  recall_score_ = recall_score(y[test], y_pred , average="macro")
  f1_score_ = f1_score(y[test], y_pred , average="macro")
  confmat_ = confusion_matrix(y[test],y_pred, labels=[0, 1, 2, 3, 4, 5, 6])
  #confusion_matrix(y_true, y_pred)
  print(precision_score_)
  print(recall_score_)
  print(f1_score_)
  print(confmat_)
  prec_per_fold.append(precision_score_)
  recall_per_fold.append(recall_score_)
  f1_per_fold.append(f1_score_)
  confmat_per_fold = np.add(confmat_per_fold, confmat_) 
  predictTimeTotal += end - start;
 

  # Increase fold number
  fold_no = fold_no + 1

print("acc: ", np.mean(acc_per_fold))
print("prec: ", np.mean(prec_per_fold))
print("recall: ", np.mean(recall_per_fold))
print("f1: ", np.mean(f1_per_fold))
print("confusion matrix:")
print(confmat_per_fold)
print("predictTimeTotal: ", predictTimeTotal)

In [ ]:
#check am i running at GPU
import tensorflow as tf
tf.test.gpu_device_name()